<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train5_rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:

train5 = pd.read_csv(r'/content/train_data5.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [ ]:

train5['Target'] = train5['Target'].replace(['Others'],'Invalid')


In [ ]:
train5['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [ ]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train5_Xtfidf = tfidf_vect.fit_transform(train5['Sentence'])


In [ ]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [ ]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [ ]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [ ]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [ ]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [ ]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train5_Xtfidf,train5['Target'],test_size=0.26,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6736725663716814
                  precision    recall  f1-score   support

        Analysis       0.53      0.33      0.40       126
      Conclusion       0.68      0.42      0.52        45
           Facts       0.73      0.89      0.80       483
         Invalid       0.52      0.56      0.54       133
           Issue       0.61      0.48      0.53        48
Rule/Law/Holding       0.68      0.30      0.42        69

        accuracy                           0.67       904
       macro avg       0.63      0.50      0.54       904
    weighted avg       0.66      0.67      0.65       904



In [ ]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [ ]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 3, 3, 2, 2, 1, 3, 0, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3, 3, 2, 2, 2, 2, 2, 5, 2, 2, 2, 3, 2, 2]
[0.9974351165201536, 0.9849854129572863, 0.9987429192192518, 0.9988570982054542, 0.981858520777507, 0.9920805629796468, 0.9980249751934395, 0.9959672487801902, 0.9981361504969356, 0.9822834001381117, 0.9961023002134669, 0.9852487482858603, 0.9968359323943297, 0.9830752169851161, 0.9974123427395627, 0.9967533648437162, 0.9979840829107637, 0.9982380275903336, 0.9978262652774287, 0.9950131913547647, 0.9917697524846761, 0.9980591016731087, 0.9979231048903597, 0.9991188293859796, 0.9844541354505655, 0.9980565502644747, 0.9905395116061392, 0.9949160366645681, 0.9945904164032532, 0.9957988591557267, 0.9919535017514605, 0.983643430687629, 0.9918304382390785, 0.9816233141837276, 0.994216575908202, 0.9911618696272662, 0.9855371645277154, 0.995004122237642, 0.9896432934347913, 0.9813066882529359, 0.99685326645893

In [ ]:
unlabel_1=unlabel_1.loc[pos,:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
0,2173,darden sr testified âœi saw staggering â prett...,Facts
2,525,85 22 sw 140 wherein accused acting upon advic...,Rule/Law/Holding
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts
6,324,defendant argues impossible reconcile plaintif...,Facts


In [ ]:
frame_1 = [train5,unlabel_1]
train5_1 = pd.concat(frame_1)
len(train5_1)

3538

In [ ]:
x_train_1 = tfidf_vect.transform(train5_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train5_1['Target'],test_size=0.2,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6694915254237288
                  precision    recall  f1-score   support

        Analysis       0.50      0.32      0.39        92
      Conclusion       0.62      0.47      0.54        32
           Facts       0.72      0.88      0.79       376
         Invalid       0.61      0.59      0.60       112
           Issue       0.59      0.37      0.46        43
Rule/Law/Holding       0.60      0.34      0.43        53

        accuracy                           0.67       708
       macro avg       0.61      0.49      0.53       708
    weighted avg       0.65      0.67      0.65       708



In [ ]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3, 3, 2, 3, 1, 2, 2, 2, 2, 1, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 2, 3, 1, 2, 2, 2, 5, 3, 4, 2]
[0.9988648531822005, 0.9990835119616182, 0.993483092293512, 0.9951407962407568, 0.9980905664117989, 0.9978389898116025, 0.9989986059076981, 0.9951887309525308, 0.9987357712457249, 0.997739732496616, 0.995940697595126, 0.9895716720364978, 0.9880253417464252, 0.988171522410247, 0.9964032669015249, 0.998553159284933, 0.9976920723924834, 0.9990425293095047, 0.9812418073841982, 0.983178447582084, 0.9988805849442117, 0.9879421190596682, 0.998958935821447, 0.9954210422367592, 0.9972444026096227, 0.9985602460335868, 0.9902684488557479, 0.9893815955611349, 0.9829222075060533, 0.9988962866926716, 0.9948786139440743, 0.9888695868992701, 0.9989661857073031, 0.999012076239854, 0.9987107074328126, 0.998907845201117, 0.9979180277840713, 0.9965738011373132, 0.9957012205250311, 0.9950404868947692, 0.9961189150276856, 0.9990710426513665, 0.991095229

In [ ]:
unlabel_2 = unlabel_2.loc[pos,:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
6,570,appellant performed work asked 7500 money ther...,Facts
7,3150,finding door facing street seen man walking pr...,Facts


In [ ]:
frame_2 = [train5_1,unlabel_2]
train5_2 = pd.concat(frame_2)
len(train5_2)

3590

In [ ]:
x_train_2 = tfidf_vect.transform(train5_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train5_2['Target'],test_size=0.27,random_state=2)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.622680412371134
                  precision    recall  f1-score   support

        Analysis       0.51      0.34      0.40       158
      Conclusion       0.60      0.36      0.45        50
           Facts       0.67      0.88      0.76       474
         Invalid       0.50      0.48      0.49       155
           Issue       0.73      0.30      0.43        73
Rule/Law/Holding       0.57      0.33      0.42        60

        accuracy                           0.62       970
       macro avg       0.60      0.45      0.49       970
    weighted avg       0.61      0.62      0.60       970



In [ ]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 3, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 1, 5, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 3, 2, 0, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2]
[0.9968447644903444, 0.9881617021310508, 0.9963993281129763, 0.9975626889969225, 0.9975325637249343, 0.9922705285483965, 0.998591293030565, 0.9980998001524511, 0.9979032140703138, 0.9891796148682584, 0.9985073182544835, 0.9904854395841276, 0.9987186124944049, 0.9988985662096763, 0.9983674074444568, 0.99449094494172, 0.9931541226175103, 0.9985214129991613, 0.9840123575774601, 0.9970095545507218, 0.9968249768122692, 0.9983079872264637, 0.993999250497221, 0.9891896782500978, 0.9981873241096745, 0.9935422926107838, 0.9853283531255927, 0.9978512782951895, 0.9982441557205202, 0.9981397970578788, 0.9833653103677182, 0.9980798320714049, 0.9931659936762348, 0.997535484310675, 0.9984623846952655, 0.9828963859642126, 0.9985673709625408, 0.9907373626508638, 0.9988170601306225, 0.99767303100420

In [ ]:
unlabel_3=unlabel_3.loc[pos,:]
frame_3 = [train5_2,unlabel_3]
train5_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train5_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train5_3['Target'],test_size=0.2,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6420765027322405
                  precision    recall  f1-score   support

        Analysis       0.43      0.25      0.31        97
      Conclusion       0.77      0.49      0.60        41
           Facts       0.70      0.87      0.78       388
         Invalid       0.51      0.53      0.52       116
           Issue       0.55      0.33      0.41        36
Rule/Law/Holding       0.58      0.28      0.38        54

        accuracy                           0.64       732
       macro avg       0.59      0.46      0.50       732
    weighted avg       0.62      0.64      0.62       732



In [ ]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[3, 3, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 3, 2, 3, 2, 2, 5, 3, 0, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2]
[0.9989997014515672, 0.9832826140812423, 0.9980801887955181, 0.9959712436093182, 0.9986992868000204, 0.9968740198324565, 0.9989145947367245, 0.9987739464846187, 0.9981435551551835, 0.9982230562643118, 0.9910543395032869, 0.9990400896153138, 0.9986951964240591, 0.99857273026617, 0.9986656266802907, 0.9922194575268803, 0.9915943624291753, 0.9838149789298924, 0.9989037269537091, 0.9982144890994402, 0.9988590628019621, 0.9944943624583585, 0.9975409363640374, 0.9987998602399485, 0.998703290278942, 0.998482243079255, 0.9985299960474084, 0.9956354329257422, 0.9872620724199714, 0.9981199138797074, 0.9885216774409423, 0.9963890226852273, 0.9972803845600668, 0.9970447425272411, 0.9951587860055271, 0.9959008594682343, 0.9978138108126802, 0.9865293315913453, 0.9918343383316359, 0.9925469568575522, 0.9990680846450141, 0.998

In [ ]:
unlabel_4=unlabel_4.loc[pos,:]

In [ ]:
frame_4 = [train5_3,unlabel_4]
train5_4 = pd.concat(frame_4)
len(train5_4)
x_train_4 = tfidf_vect.transform(train5_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train5_4['Target'],test_size=0.27,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6603585657370518
                  precision    recall  f1-score   support

        Analysis       0.51      0.31      0.38       143
      Conclusion       0.62      0.38      0.48        52
           Facts       0.71      0.88      0.79       532
         Invalid       0.53      0.55      0.54       151
           Issue       0.73      0.37      0.49        60
Rule/Law/Holding       0.63      0.36      0.46        66

        accuracy                           0.66      1004
       macro avg       0.62      0.48      0.52      1004
    weighted avg       0.65      0.66      0.64      1004



In [ ]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 3, 3, 2, 3, 2, 5, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 0, 2, 2, 2, 3, 3, 2, 3, 3, 2, 2, 2, 2, 2, 2, 3, 2, 2, 5, 2, 2, 2, 2, 2, 2, 2, 5, 2, 2, 2, 2, 2, 5, 2, 3, 0, 3, 2]
[0.9986405507633245, 0.9922588215165963, 0.998709938813392, 0.9975702785337662, 0.9830105407178434, 0.9982227340544272, 0.9914501653367905, 0.9984528123075803, 0.998709938813392, 0.9984184044096854, 0.9986704040323738, 0.992085212189135, 0.9880316043823699, 0.9936890024159459, 0.9838922856493426, 0.9986795205091186, 0.9910996995662048, 0.9936833588055402, 0.9863198672501164, 0.9981002981878904, 0.9812767626538973, 0.9879531336489391, 0.9885917058830295, 0.9851776825519539, 0.9962486638838248, 0.9985589664757524, 0.998510740568555, 0.9980047840247314, 0.998709938813392, 0.998023730379035, 0.9971876401946791, 0.9877500672336478, 0.9923553405671663, 0.9984728493502815, 0.9954412504752622, 0.9957789943182355, 0.9979233827057217, 0.9970006887706353, 0.9966599471824183, 0.9829215509351865, 0.9942231436910117, 0

In [ ]:
unlabel_5 = unlabel_5.loc[pos,:]

In [ ]:
frame_5 = [train5_4,unlabel_5]
train5_5 = pd.concat(frame_5)
len(train5_5)
x_train_5 = tfidf_vect.transform(train5_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train5_5['Target'],test_size=0.27,random_state=42)
support = RandomForestClassifier()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6542605288932419
                  precision    recall  f1-score   support

        Analysis       0.45      0.33      0.38       143
      Conclusion       0.71      0.39      0.51        56
           Facts       0.74      0.85      0.79       546
         Invalid       0.47      0.60      0.53       147
           Issue       0.76      0.45      0.57        55
Rule/Law/Holding       0.56      0.27      0.36        74

        accuracy                           0.65      1021
       macro avg       0.61      0.48      0.52      1021
    weighted avg       0.65      0.65      0.64      1021



In [ ]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.7197696737044146


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(test['Target'],test_pred,average='macro'))

(0.7404690039050376, 0.625265717028671, 0.664069285958531, None)
